In [13]:
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
import requests
from langchain_core.tools import InjectedToolArg
from typing import Annotated

Task- Create a currency converter

In [14]:
# tool to fetch conversion data
@tool
def currency_conversion_factor(base_curr: str, target_curr: str )->float:
    """
    Takes two inputs- base_currency and target_currency and return fetch the details from one conversion to another
    """
    API_key="3e385583037b97e663fdd071"
    url=f"https://v6.exchangerate-api.com/v6/{API_key}/pair/{base_curr}/{target_curr}"
    response=requests.get(url)
    return response.json

#tool to convert the currency
@tool
def convert(base_currency_value:float, conversion_rate: Annotated[float, InjectedToolArg])-> float:
    """
    Takes base_currency_value and conversion rate to calculate target currency value
    """
    return base_currency_value*conversion_rate

# conversion_rate: Annotated[float, InjectedToolArg] this is to ensure that LLM doesn't set it's value based on its previous knowledge and let the user to set it.

In [12]:
# BINDING
llm=ChatOpenAI()
llm_with_tools=llm.bind_tools([currency_conversion_factor, convert])

In [15]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

ai_message=llm_with_tools.invoke(messages)

messages.append(ai_message)

APITimeoutError: Request timed out.

In [ ]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = currency_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [ ]:
llm_with_tools.invoke(messages).content